In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from bosonStar import *

massInCU = 2e-77
aInCU = 1e-74
totalMassInCU = 0.1
dtInCU = 10000.0
bigT = 4000

In [ ]:
allThings = outputProfile(massInCU, aInCU, totalMassInCU)
#Hot Fix
x = allThings["x"][1::2]
psi = allThings["profile"][1::2]

In [ ]:
def lengthCodeUnitToSI(cu):
    return cu/6.77193e-6/100.0

def lengthSIToPlanckUnit(SI):
    return SI/1.616255e-35

def densityCodeUnitToSI(cu):
    return cu*1000.0/1.619e-18

def densitySIToPlanckUnit(SI):
    return SI/5.155e96

def massCodeUnitToSI(cu):
    return cu/5.02788e-34/1000

def massSIToPlanckUnit(SI):
    return SI/2.176434e-8

def timeCodeUnitToSI(cu):
    return cu/2.03017e5

def timeSIToPlanckUnit(SI):
    return SI/5.391247e-44

In [ ]:
totalMass = massSIToPlanckUnit(massCodeUnitToSI(totalMassInCU))
m = massSIToPlanckUnit(massCodeUnitToSI(massInCU))
a = lengthSIToPlanckUnit(lengthCodeUnitToSI(aInCU))
N = totalMass/m

In [ ]:
HBAR = 1.0
G = 1.0
c = 1.0

In [ ]:
Ra = np.sqrt(a/m/m/m)
omegaA = m*m/np.abs(a)
epsilon = 1.0/omegaA/m/Ra/Ra
a0 = np.sqrt(1.0/omegaA/m)
delta = 4.0*np.pi*a*N/a0
GM = totalMass/Ra/Ra/Ra/omegaA/omegaA

print("R:", Ra)
print("Omega:" ,omegaA)
print("Epsilon (Should be 1):", epsilon)
print("Delta:", delta)
print("New G:", GM)

In [ ]:
x = lengthSIToPlanckUnit(lengthCodeUnitToSI(x))/Ra
x = np.concatenate((np.flip(-x), x))

psi = np.sqrt(densitySIToPlanckUnit(densityCodeUnitToSI(psi))/totalMass)*np.power(Ra, 3.0/2.0)
psi = np.concatenate((np.flip(psi), psi))
normalization = np.trapz(2.0*np.pi*x*x*np.absolute(psi)*np.absolute(psi), x)
psi = psi/normalization
psi = psi.astype(complex)

for index, value in enumerate(psi):
    if(value<1e-8):
        psi[index] = 0
psi = psi.astype(complex)

size = len(psi)
dt = timeSIToPlanckUnit(timeCodeUnitToSI(dtInCU))*omegaA

print(psi)
print(dt)

In [ ]:
dx = x[1]-x[0]
T = np.zeros((size,size))
TSph = np.zeros((size,size))
V = np.zeros((size,size))
Q = np.zeros((size,size))
V2Grav = np.zeros((size,size))
I = np.identity(size)
I = I.astype(complex)

phi = -np.exp(-x*x)/1000.0
phi[0] = 0.0
phi[size-1] = 0.0

In [ ]:
def setupT():
    global T, TSph
    for i in range(size):
        T[i][i] = -2.0
        if(i!=size-1):
            T[i][i+1] = 1.0
            TSph[i][i+1] = 1.0/2.0/x[i]
        if(i!=0):
            T[i][i-1] = 1.0
            TSph[i][i-1] = -1.0/2.0/x[i]

    T = -1.0/2.0/dx/dx*T
    TSph = -1.0/dx*TSph
    T = T.astype(complex)
    TSph = TSph.astype(complex)

def setupQ():
    global Q
    for i in range(size):
        if(np.absolute(psi[i]) >= 1e-8):
            Q[i][i] = delta*np.absolute(psi[i])*np.absolute(psi[i])

    Q = Q.astype(complex)

def setupV2Grav():
    global V2Grav, phi, psi

    phi[0] = 0.0
    phi[size-1] = 0.0

    VGrav = np.zeros((size,size))
    VGrav2 = np.zeros((size))
    for i in range(size):
        if(i!=size-1 and i!=0):
            VGrav[i][i+1] = 1.0/2.0 + 1.0/2.0*dx/x[i]
            VGrav[i][i-1] = 1.0/2.0 - 1.0/2.0*dx/x[i]
        if(np.absolute(psi[i]) >= 1e-8):
            VGrav2[i] = 2.0*np.pi*dx*dx*np.absolute(psi[i])*np.absolute(psi[i])*GM

    toler = 3e-8
    count = 0

    while(True):
        phiNew = np.dot(VGrav, phi) - VGrav2
        error = np.abs((phiNew-phi)/phi)
        phi = phiNew
        count = count + 1
        if(np.nanmax(error[error!=np.inf])<toler):
            print(count)
            break
        
    for i in range(size//2):
        if(np.absolute(psi[i]) >= 1e-8):
            V2Grav[i][i] = phi[size-i-1]
            V2Grav[size-i-1][size-i-1] = phi[size-i-1]
        #Regulate
        phi[i] = phi[size-i-1]

    V2Grav.astype(complex)

def setupU1U2():
    global U1, U2
    U1 = I + 1.0j*dt/2.0/HBAR*(T+TSph+Q+V2Grav)
    U2 = I - 1.0j*dt/2.0/HBAR*(T+TSph+Q+V2Grav)
    #U1 = I + 1.0j*dt/2.0*(T+TSph+V2Grav)
    #U2 = I - 1.0j*dt/2.0*(T+TSph+V2Grav)

In [ ]:
setupT()

In [ ]:
setupU1U2()
hello = []
ts = []
oldGM = GM
newGM = GM*1.1
for t in range(bigT):
    #Pertubation
    if(t==0):
        GM = newGM
    if(t==10):
        GM = oldGM


    print(t, t*dt, np.absolute(psi[size//2]))
    if(GM == oldGM):
        hello.append(np.absolute(psi[size//2])*np.absolute(psi[size//2]))
        ts.append(t*dt)

    setupQ()
    setupV2Grav()
    setupU1U2()
    psi = np.linalg.solve(U2, np.dot(U1,psi))

    #Regulate
    for i in range(size//2):
        psi[i] = psi[size-i-1]

    V2Grav.astype(complex)

    if t%100 == 0:
        plt.plot(x, np.absolute(psi)*np.absolute(psi))

plt.xlabel(r"$r$ (dimensionless)")
plt.ylabel(r"$\left | \psi^2 \right |$ (dimensionless)")
plt.show()

In [ ]:
plt.plot(ts, hello)
plt.xlabel(r"$t$ (dimensionless)")
plt.ylabel(r"$\left | \psi^2 \right |$ (dimensionless)")
plt.show()

In [ ]:
sp = np.fft.fft(hello)
freq = np.fft.fftfreq(len(hello), d=dt)*omegaA/5.391247e-44

plt.plot(freq, np.absolute(sp))
plt.xlim(0.01,0.7)
plt.ylim(0.0,0.015)
plt.xlabel("f (1/s)")

In [ ]:
for i in range(len(freq)):
    print(freq[i], np.absolute(sp[i]))

Middle
0.071-0.081			M1 (M2: 0.142-0.162, M3: 0.213-0.243, M4: 0.284-0.324, M5: 0.355-0.405)
0.132-0.137			M1?
0.158-0.163			M2
0.193-0.204			M1?
0.239-0.249			M3
0.295-0.305			M4
0.361-0.371			M5

0.076-0.086			M1
0.132-0.142			M1?
0.158-0.168			M2

Small
0.097-0.107		M1
0.163-0.173		M1?
0.214-0.224		M2
0.270-0.280		M1?
0.341-0.351		M3
0.432-0.443		M4
0.534-0.544		M5
0.651-0.661		M6